In [1]:
%reload_ext autoreload
%autoreload 2

from llm_blocks import blocks

In [26]:
registry = blocks.create_block_registry()

template_block = registry.create("template", template="Hello, {name}")
chat_block = registry.create("chat")
generic_block = registry.create("generic")